In [1]:
import time
import datetime
import requests
import pandas as pd
from bs4 import BeautifulSoup

import pricemap

import plotly.graph_objects as go
import numpy as np
import chart_studio
import chart_studio.plotly as py

from apscheduler.schedulers.blocking import BlockingScheduler

user_name = "ricardo.delacruz"
api_key = "20Ilxq1trhOcXgRcwYdZ"
chart_studio.tools.set_credentials_file(username=user_name, api_key=api_key)

In [2]:
def create_fig(retailers, skus, title, price_evolution_data):   
    ## create traces
    fig = go.Figure()

    ## navigate for each retail and sku
    for retail, sku in zip(retailers, skus):
        ## select the sku and retail
        query = price_evolution_data.loc[(price_evolution_data['sku'] == sku) & (price_evolution_data['retail'] == retail)]
        price = query["price_float"] ## get the price
        date = query["date_time"] ## get the date
        
        fig.add_trace(go.Scatter(x=date.values,
                                 y=price.values,                                
                                 name=retail))
        
    fig.update_traces(mode='lines+markers')
    fig.update_layout(title={
                            'text': title,
                            'y':0.9,
                            'x':0.5,
                            'xanchor': 'center',
                            'yanchor': 'top'})
    
    return fig

# Empieza el codigo

1. se debe hacer en funcion como el codigo original (colocar shed)
6. Implementar que bote una alerta cuando no se pueda hacer SCRAPING
8. Hacer estadisticos basicos de los productos
9. Hacer q el codigo pueda ser migrado
10. scrap retailers es version premium, el de nosotros se trabajaria con una keyword preferiblemente con Selenium

EL PRECIO SE GUARDA EN price_evolution_data No en retail

In [3]:
#Lee los archivos .csv
price_evolution_data = pd.read_csv("price_evolution.csv")
retail_data = pd.read_csv("retail_data_final.csv")

In [ ]:
#Se aplica get_price_retail de pricemap.py a los datos de retail_data (EL código demora)
df = pd.DataFrame(columns=price_evolution_data.columns)
df["sku"], df["price"],df["price_tienda"],df["price_tarjeta"], df["retail"], df["date"], df["time"]= zip(*retail_data.apply(lambda x: pricemap.get_price_retail(x["uri"], x["retail"], x["sku"]), axis=1))

In [6]:
df.tail() #EMPECE 10:42 TERMINO 12:13 (90 minutos con 4341 productos, masomenos 50 productos x minuto)

,sku,price,price_tienda,price_tarjeta,retail,date,time
6519,227937,None,None,None,metro,2022-08-22,12:00
6520,945117,None,None,None,metro,2022-08-22,12:00
6521,727401,14.99,None,None,metro,2022-08-22,12:00
6522,545313,13.99,None,None,metro,2022-08-22,12:00
6523,903181,47.79,None,None,metro,2022-08-22,12:00


In [7]:
#Se guardan/actualizan en .csv los datos dentro de price_evolution_data
price_evolution_data = pd.concat([price_evolution_data, df], axis=0, ignore_index=True)
#price_evolution_data.dropna(how="any", inplace=True)
price_evolution_data.to_csv("price_evolution.csv", index=False)
price_evolution_data

,sku,price,price_tienda,price_tarjeta,retail,date,time
0,20147381,NaN,NaN,NaN,plaza_vea,2022-08-21,21:00
1,20147047,27.99,29.7,NaN,plaza_vea,2022-08-21,21:00
2,20136609,7.59,NaN,NaN,plaza_vea,2022-08-21,21:00
3,20136607,7.59,NaN,NaN,plaza_vea,2022-08-21,21:00
4,985158,11.8,NaN,NaN,plaza_vea,2022-08-21,21:00
...,...,...,...,...,...,...,...
13053,227937,None,None,None,metro,2022-08-22,12:00
13054,945117,None,None,None,metro,2022-08-22,12:00
13055,727401,14.99,None,None,metro,2022-08-22,12:00
13056,545313,13.99,None,None,metro,2022-08-22,12:00


In [8]:
#Consigue el tamaño
size = price_evolution_data.shape

In [9]:
##Guarda en variable solo los datos de la ultima semana
seven_days_ago = pricemap.get_time(today=False)
price_evolution_data = price_evolution_data.loc[price_evolution_data['date'] >= seven_days_ago[0]]   

In [10]:
#Se limpia los datos y se pasan al formato indicado
#price_evolution_data["price_float"] = price_evolution_data["price"].apply(lambda x: float(str(x).split(" ")[0].replace(',','')))
price_evolution_data["price_float"] = price_evolution_data["price"].apply(lambda x: float(str(x).replace(',','')) if(x is not None) else "None")

price_evolution_data["date_time"] = price_evolution_data[["date", "time"]].apply(lambda row: " ".join(row.values), axis=1)


/home/labella/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/labella/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [32]:
%%time
retailers = ["wong", "metro", "plaza_vea", "tottus", "vivanda"]
skus = ["59539001", "59539001", "497497", "inca-kola-gaseosa-10174358/p/", "497497"]
title = 'INKA COLA 500ML'
fig = create_fig(retailers, skus, title, price_evolution_data)
#py.plot(fig, filename="inka_cola_500ml", auto_open=False)

CPU times: user 113 ms, sys: 78 µs, total: 113 ms
Wall time: 114 ms


In [17]:
fig

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


In [33]:
## Coca Cola 500ml
retailers = ["wong", "metro", "plaza_vea", "tottus", "vivanda"]
skus = ["3814", "3814", "281027", "coca-cola-gaseosa-no-retornable-10164146/p/", "281027"]
title = 'COCA COLA 500ML'
fig = create_fig(retailers, skus, title, price_evolution_data)
#py.plot(fig, filename="coca_cola_500ml", auto_open=False)

In [34]:
fig

In [35]:
%%time
## arroz extra costeño bolsa 5kg
retailers = ["wong", "metro", "plaza_vea", "tottus", "vivanda"]
skus = ["14974", "14974", "641425", "costeno-arroz-extra-10472604/p/", "641425"]
title = 'ARROZ EXTRA COSTEÑO BOLSA 5Kg'
fig = create_fig(retailers, skus, title, price_evolution_data)
#py.plot(fig, filename="arroz_extra_costeno_bolsa_5kg", auto_open=False)

CPU times: user 224 ms, sys: 3.96 ms, total: 228 ms
Wall time: 227 ms


In [36]:
fig